In [1]:

# buildAnki.ipynb
# build MWALED import file for Anki


import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs


#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest


# remember to change these values
wlflag = 1 # 1 for a wordlist; 0 for mwaled
wlname = 'chocolate' # wordlist file name will be wlname+List.txt, which has been built by hand using UNIX commands
excluflag = 0 # 1: exclude words in exclu.txt; 0: not 


if wlflag == 0:
    # for all words of mwaled
    fdict = open('anki\\mwaledAnki.txt', 'w')
else:
    # for a word list
    fdict = open('anki\\' + wlname + 'Anki.txt', 'w')
    words = []
    for line in open('anki\\' + wlname + 'List.txt', 'r'):
        words.append(line.strip())
        
    exclus = []
    if excluflag == 1:        
        for line in open('anki\\exclu.txt', 'r'):
            exclus.append(line.strip())
    #print words
    
results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1, "audiofile":1})
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)


# Produce Anki note fields and tags:
for result in results:
    # Anki note field: head word
    word = result.get("hw")    
    if word:        
        if wlflag == 1 and word not in words: continue
        if word in exclus: continue
        print word
        fdict.write(word.encode('ascii', 'ignore'))
    fdict.write("\t") 
    
    
    # Anki note field: Pronunciation in ASCII IPA symbols
    prns = result.get("prn")
    if prns:
        for prn in prns:
            fdict.write("/" + prn + "/<br>")
    fdict.write(" \t")
    
    
    # Anki note field: Audio
    multiFlag = 0
    audiofile = result.get("audiofile")
    if word: 
        if audiofile: 
            fdict.write("[sound:" + audiofile + "]")
            multiFlag = 1
        else: fdict.write("[sound:" + word.encode('ascii', 'ignore') + ".wav]")
    fdict.write(" \t")
    
    
    # Anki note field: Recall Hint  
    sublists = result.get("sublist")
    if sublists:
        cnt = 1
        for sublist in sublists: #sublist is dictionary
            if sublist.has_key("def"):
                fdict.write(str(cnt) + '<font color="blue">-def: </font>' + sublist["def"].encode('ascii', 'ignore') + "<br>")
            #if sublist.has_key("idpv"): 
            #    fdict.write('<font color="red">*</font> ' + sublist["idpv"].encode('ascii', 'ignore') + "<br>")            
            if cnt%2 == 0: fdict.write('<br>')
            cnt += 1
            
    fdict.write("<br>")
    idpvs = result.get("idpvs")
    if idpvs:
        for idpv in idpvs:
            fdict.write(idpv.encode('ascii', 'ignore') + "<br>")                
    fdict.write(" \t") # add one space before tab to separate tabs if a field is empty.
    
    
    # Anki note field: Snippet Producing    
    #if alles: 
    #    for es in alles:
    #        fdict.write(es.encode('ascii', 'ignore') + "<br>")
    if sublists:
        cnt = 1
        for sublist in sublists: #sublist is dictionary
            if sublist.has_key("es"):
                for es in sublist["es"]:
                    fdict.write(str(cnt) + '<font color="blue">-es: </font>' + es.encode('ascii', 'ignore') + "<br>")
            if sublist.has_key("idpv"): 
                fdict.write('<font color="red">*idpv*</font> ' + sublist["idpv"].encode('ascii', 'ignore') + "<br>")
            if cnt%2 == 0: fdict.write('<br>')
            cnt += 1
    fdict.write(" \t")
   
    
    # Anki note field: Footnote    
    fdict.write(" \t")    

    # Anki tags
    alles = result.get("alles")
    if alles: totales = len(alles)
    else: totales=0
        
    if totales >= 100: nes = 1
    elif totales >= 50: nes = 2
    elif totales >= 10: nes = 3
    elif totales >=  2: nes = 4
    else: nes = 5
        
    if wlflag == 1: fdict.write(wlname + "\n")
    elif multiFlag == 1 and word != 'con' : fdict.write("msound mwaled level-" + str(nes) + "\n") # msound -- multiple sounds
    else: fdict.write("mwaled level-" + str(nes) + "\n")
    
fdict.close()    




yeah
subject
sure
sure
scenario
subject
subject
process
process
quiet
quiet
quiet
really
ideal
ideal
ideally
marry
of
our
pedestrian
pedestrian
chocolate
daughter
downtown
downtown
generosity
actually
aroma
banana
bother
bother
